# PydanticOutputParser

`PydanticOutputParser`는 언어 모델의 출력을 **구조화된 정보**로 변환하는 데 도움을 주는 클래스입니다. 이 클래스는 단순 텍스트 응답 대신 **명확하고 체계적인 형태로 필요한 정보를 제공**할 수 있습니다.

이 클래스를 활용하면 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여 정보를 더 쉽게 처리하고 활용할 수 있습니다.

## 주요 메서드

`PydanticOutputParser` (대부분의 OutputParser에 해당)에는 주로 **두 가지 핵심 메서드**가 구현되어야 합니다.

- **`get_format_instructions()`**: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이 지침은 언어 모델이 출력을 구조화하고 특정 데이터 모델에 맞게 변환하는 데 매우 중요합니다.
- **`parse()`**: 언어 모델의 출력(문자열로 가정)을 받아 이를 특정 구조로 분석하고 변환합니다. Pydantic과 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다.

## 참고 자료

- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/)

> 

In [ ]:
from dotenv import load_dotenv
from langchain.chains.summarize.refine_prompts import prompt_template

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

다음은 이메일 본문 예시입니다.


In [ ]:
email_conversation = """From: 이유진 (leeyu@spectra.co.kr) 
To: 한경만 (kmhan@spectra.co.kr)
Date: 2024년 2월 22일 (목) 오후 3:27
Subject: [협조] 2월 법인카드 영수증 제출 기한 안내_3월 5일(화)까지

안녕하세요, 경영지원팀 이유진입니다.
2월 비용 마감을 위해 비용(법인카드) 제출 기한을 안내 드립니다.

  1. 제출 기한: ~2024년 3월 5일(화)까지
  2. 안내 대상: 법인카드 소유자
  3. 제출 방법: 플렉스 - 워크플로우 - 문서 작성하기 - 비용신청서 작성 및 영수증 제출
  4. 기타 사항
    a. 상주 PJ의 경우에는 비용 먼저 청구해 주시고 영수증은 추후 제출해주시기 바랍니다.
    b. 법인카드 소유자와 실제 사용자가 다른 경우, 사용자가 비용 신청서 작성해주시고 비용신청서 상의 법인카드 소유자에 해당 카드 소유자명을 적어주시기 바랍니다.

상기 내용 확인하시고 제출 기한 내 제출 부탁드립니다. :)

감사합니다. :)
이유진 드림
"""

출력 파서를 사용하지 않는 경우 예시

In [ ]:
from share.messages import stream_response
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용 중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

In [ ]:
print(output)

위와 같은 이메일 내용이 주어졌을 때 아래의 Pydantic 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱해 보겠습니다.

참고로, Field 안에 `description` 은 텍스트 형태의 답변에서 주요 정보를 추출하기 위한 설명입니다. LLM 이 바로 이 설명을 보고 필요한 정보를 추출하게 됩니다. 그러므로 이 설명은 정확하고 명확해야 합니다.

In [ ]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [ ]:
# instruction 을 출력합니다.
print(parser.get_format_instructions())

프롬프트를 정의합니다.

1. `question`: 유저의 질문을 받습니다.
2. `email_conversation`: 이메일 본문의 내용을 입력합니다.
3. `format`: 형식을 지정합니다.


In [ ]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

다음으로는 Chain 을 생성합니다.


In [ ]:
# chain 을 생성합니다.
chain = prompt | llm

체인을 실행하고 결과를 확인합니다.


In [ ]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용 중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)

마지막으로 `parser`를 사용하여 결과를 파싱하고 `EmailSummary` 객체로 변환합니다.


In [ ]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
structured_output

## parser 가 추가된 체인 생성

출력 결과를 정의한 Pydantic 객체로 생성할 수 있습니다.

In [ ]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser

In [ ]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

## with_structured_output()

`.with_structured_output(Pydantic)`을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환할 수 있습니다.

In [ ]:
llm_with_structured = ChatOpenAI(
    temperature=0, model_name="gpt-4o-mini"
).with_structured_output(EmailSummary)

In [ ]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structured.invoke(email_conversation)
answer

**참고**

한 가지 아쉬운 점은 `.with_structured_output()` 함수는 `stream()` 기능을 지원하지 않습니다.

------------------------
이렇게 사용하면 streaming을 사용할 수 있다. (by kmhan)

In [ ]:
# summary를 젤 아래로 이동한다. (Streaming 대상)
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")
    summary: str = Field(description="메일 본문을 요약한 텍스트")

In [ ]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}
"""
)

In [ ]:
llm = ChatOpenAI(
    temperature=0, model_name="gpt-4o-mini"
)
llm_with_structured = llm.with_structured_output(EmailSummary)

In [ ]:
chain = prompt | llm_with_structured

In [ ]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = chain.stream({
    "email_conversation": email_conversation,
    "question": "이메일 내용중 주요 내용을 추출해 주세요.",
})

In [ ]:
for token in answer:
    print("\n")
    print(token.model_dump_json(), flush=True)